Import our dependencies and the Minitorch from my github

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import requests
import sys
import os
from tqdm import tqdm


from engine.minitorch import Tensor,optim

I will put a seed right now for the reproducibility of the project.

In [3]:
np.random.seed(3329)

This are the variables that we need in order to create the data folder with the dataset inside,because I can't upload the repository with the dataset of Shakespeare pre-charged.

In [4]:
data_directory = "data"
file_path = os.path.join(data_directory, "input.txt")
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"

This will create a data folder if it doesn't exists

In [5]:
if not os.path.exists(data_directory):
    
    os.makedirs(data_directory)

if the inside of the data folder that we just created is empty we throw a request to the url for the dataset of Shakespeare of the github of Karpathy,raise for status to inform us what happened with the request and then open it indicating to the computer that need to transform it binary to avoid missinterpretations of characters and then finally write it down to the "input.txt" inside our data folder

In [6]:
if not os.path.exists(file_path):
    
    response = requests.get(url)
    
    response.raise_for_status()


    with open(file_path, "wb") as f:

        f.write(response.content)

        

Here what we do is open the input.txt of Shakespeare preaviously stored in the data folder and read it,the encoding utf-8 is the standard to read special characters like emojis,commas,dots,etc.\
Then the text variable is just all that concatenated ,like a list of all Shakespeare.

In [7]:
with open(file_path, 'r', encoding='utf-8') as f:

    text = f.read()

First we convert the text into a set,this will give us every character inside the text.\
Then we transform that set into a list to be able to sorted in order that the same character never change its index.\
Lastly we can see the len of the vocabulary and print it.

In [8]:
chars = sorted(list(set(text)))
vocab_size = len(chars)


print(chars)
print(vocab_size)

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
65


To tokenize them we just create two dictionaries, String to Integer(stoi) and Integer to String(itos).

One will have as key_value the character and as the value the index of that character in the list of chars.

And the other one will have its key_value the index and as value the character of that index in the list of chars.

This is exactly what we want because we use the list of characters and its indexes to transform them into integers and vice versa.

In [9]:
stoi = {ch:i for i,ch in enumerate(chars)}

itos = {i:ch for i,ch in enumerate(chars)}

Now here is the actual encoding and decoding.\
We gonna use lambda function because this is simple and with lambda we avoid writing a def just for this.

encoder = here we pass to lamba a string(s) then we ask for its correspondent value inside the stoi dictionary.

decoder = this will receive a list of numbers,search what character correspond to that integer inside of itos and concatenate them into a string.

In [10]:
encoder = lambda s:[stoi[c] for c in s]

decoder = lambda l: ''.join([itos[i]for i in l])

Now that we got our encoder and decoder que can transform the text into a numpy array.

In [ ]:
all_data = np.array(encoder(text), dtype=np.int64)

Here we gonna take the 90% of the total data(len of total_data * 0.9).\
And then we use that number to divide the data into train_data and test_data.

train_data being the first 90% of the data.\
test_data being the last 10% of the data.

In [ ]:
n = int(0.9 * len(all_data))

train_data = all_data[:n]
test_data = all_data[n:]

Here is the function to create batches because for training you can't pass all the data at once,explanation = Self-explanatory, just don't.

For that reason we create this function passing as arguments the split which is a string.\
The batch_size which is one of the dimensions and the block_size which is the other dimension.

First we declare the variable data, this variable will be equal to the train_data if when we pass the split(the string parameter) we said "train" otherwise the variable data will be equal to the test_data.

Now we gonna create the randomness in the batches in order to train better the model,using numpy,we tell it to give us a random number between 0 and the total of the data minus the block size so it doesn't give us por example the last 2 tokens in the data and broke our block size.\
And then we also pass the batch_size to tell numpy that we want more than a single number.

Next we gonna create our Tensors2D(block_size,batch_size).\
To create our inputs (x) we need to put them in a stack in order to create the mesh/matrix so using and excel sheet as analogy you can interpret the block size as the how large will we the rows and the batch_size as how large will be the columns(for example, if we pass block_size=8 and batch_size=4 the tensor that we gonna get is a 4x8 matrix,4 batches with a lenght of 8 values each).\
Once acquire the inputs, the targets(y) follow the same creation logic but +1.It will be the same size as the tensor "X".
Example: if tensor start with something like  43,56,21 then tensor Y will be like 56,21,33 .

Lastly we just return both the inputs and targets tensors.

In [ ]:
def get_batches(split,batch_size,block_size):

    data = train_data if split == "train" else test_data

    idx = np.random.randint(0,len(data) - block_size,batch_size)

    x = np.stack(data[i:i+block_size] for i in idx)

    y = np.stack(data[i + 1 : i + block_size + 1] for i in idx)

    return x, y

Here we have our variables to be able to create the table of embedding.\
We will use this to transform the 65 posibles tokens into vectors of random values(one vector per char) to be able to do maths operations with the vectors.

First we take the vocab_size.\
Then we assing a variable called numbers of embedding which is arbitrary,for a model like this I recommend 128 because is a tiny model,every of the 128 will give to the model context of what is about the token, for example, if the token 5 represent the letter "a" the table of embedding with each of the 128 will give us an adjective of it: "vocal","minuscule".


Then we initialize the variable table creating a tensor2D of random numbers,each row will represent 1 token(65 row because of the vocab_size) and every column is value of embedding for that token(128 embeddings per token).Creating a tensor2D of (65,128).\
We just multiply those values by the Xavier formula that is commonly used for Softmax/Sigmoid/Tanh because the randomness is too high for the model to start learning right away,it could probably(and luckily if doesn't explode)take a lot of epochs start learning.

By the way,we created this Variables outside the get_embeddings function because we need to train this random values,if we initialize them inside the function they will be created every time we call the function and never train.

In [ ]:
vocab_size = vocab_size # Just to bring it closer we already know that they are 65 characters

n_embd = 128


token_table = np.random.randn(vocab_size,n_embd) * np.sqrt(2/n_embd)

This short function is the embedding of our tokens,is short because Numpy does the hard lifting for us creating the new tensor and all that.

When we call the function we pass our batches of tokens for example (4x8) and we also pass the table of embeddings of random values(65,128).\
What Numpy does is create a new Tensor, but this tensor is a Tensor3D.\
When we said use the inputs as index of table we practically say,from the embeddings table put in a new tensor the rows of every value in the block_size of the batch_size giving us a Tensor3D of (4,8,128).\
In an excel sheet example in this new tensor we got: the 128 embeddings columns(n_embd=128) for each row of the 8 rows(block_size=8) and once we got that tensor2D we need to stack 4 tensor2D like those(batch_size=4) on top of each other.

In [ ]:
#  def get_embeddings(xb,token_table):
#      return token_table[xb]

Same as the token_table we iniliazite a tensor2D but instead of using the vocab_size we use the block_size(Also multiplying by the Xavier formula).

In [ ]:
pos_table = np.random.randn(block_size,n_embd) * np.sqrt(2/n_embd)

I encourage you to learn the theory behind this positional encoding,is facinating see how words can follow a sinusoidal wave to know time and position of each embedding(maybe in future projects I'll implement this advance,more mathematical version on a transformer,which is the original way suggested by the paper "Atenttion is all you need",extrapolation, "infinite" positions),For now I'll stick with the GPT way(Learned positional embeddings)which is just another tensor like the "embeddings_table" that add time and position with random numbers that later on the model will learn.\
Because the model without the positional encoding will think that "The dog bites the man" is the same as "The man bites the dog".


Here we take as parameters the stack batch(4,8) the token table and pos_table that we previusly have created.

For future operations I'll take the shape of xb .\
Now to make things more modular just in case that in the future I want a block_size of 16 instead of one of 8 like right now,we create an numpy array with the length of our block_size(T=8).

Then we have the token_table(65,128) but we need a tensor3D of (4,8,128 like the token_embeddings of the previous function).\
so we use the same fancy indexing trick to give us the tensor3D.(after a conclusion I find the previous functon redundant and I comment it, so it doesn't affect when running all the cells,just in case;But I will leave it right there because I see it as a sign of learn not as a mistake).

Then this next part is just to "name" put "labels" to every row in the pos_table,for example the first row of pos_table will be named position:0(this a analogy of how the model will think) and the next row position:1, this will continue until position:7 because the block_size only have 8 positions.\
This will give to the model a perception of time/space because this value will be train to tell the posible position of the token after seeing all the Shakespeare train data.

Next and finally is just the summ of the token_embd which give us "characteristics" of the token with the pos_embd that will give us "time/space" of the token,if you are wondering how two tensor with different sizes can do an addition to each other,the answer is : Because of numpy,numpy is doing the broadcasting so we don't have problems with the dimension,he es just doing again and again the addition of the tensor2D(pos_embd) the amount of batches of the tensor3D(4),so we ended up with a (4,8,128) tensor.

In [ ]:
def positional_encoding(xb,token_table,pos_table):

    B,T = xb.shape

    pos_idx = np.arange(T)

    token_embd = token_table[xb]

    pos_embd = pos_table[pos_idx]

    x = token_embd + pos_embd

    return x